In [1]:
import pandas as pd
import numpy as np

from Data_Format_Change import long_format_to_sparse, sparse_to_long_format
from mae_mse import get_mae, get_mse

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# Ensemble 

- Baseline model
- SGD(funk-svd)
- ALS(with regularization)
- SGD(svd+bias)
- CB
- tf-idf cos

# Baseline Model

In [ ]:
baseline_long_format = pd.read_csv('../data/baseline_predict_df.csv')
baseline_long_format = baseline_long_format.loc[ : , ~baseline_long_format.columns.str.contains('Unnamed')]
baseline_sparse = long_format_to_sparse(baseline_long_format, 'prediction')
baseline_sparse

# MF Model

In [ ]:
# np.save('../data/test_prediction_CF_ALS.npy', prediction)
cf_als_sparse = np.load('../data/test_prediction_CF_ALS.npy')
cf_als_sparse
rows = np.nonzero(valid_sparse_matrix)[0]
cols = np.nonzero(valid_sparse_matrix)[1]
cf_als_test = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
loc_zip = zip(rows, cols)
for loc in loc_zip:
    cf_als_test[loc] = cf_als_sparse[loc]
cf_als_test

In [ ]:
cf_sgd_sparse = np.load('../data/test_prediction_CF_SGD.npy')
cf_sgd_sparse
rows = np.nonzero(valid_sparse_matrix)[0]
cols = np.nonzero(valid_sparse_matrix)[1]
cf_sgd_test = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
loc_zip = zip(rows, cols)
for loc in loc_zip:
    cf_sgd_test[loc] = cf_sgd_sparse[loc]
cf_sgd_test

# CB Model

In [ ]:
nlp_long_format = pd.read_csv('../data/Predictions_CB_bus.csv')
nlp_sparse = long_format_to_sparse(nlp_long_format, 'prediction_ratings')
nlp_sparse

In [ ]:
cb_tfidf_long_format = pd.read_csv('../data/Predictions_CB_tfidf.csv')
cb_tfidf_sparse = long_format_to_sparse(cb_tfidf_long_format, 'prediction_ratings')
cb_tfidf_sparse

In [ ]:
cb_lsi_long_format = pd.read_csv('../data/Predictions_CB_LSI.csv')
cb_lsi_long_format.head()
cb_lsi_sparse = long_format_to_sparse(cb_lsi_long_format, 'pred_lsi')
cb_lsi_sparse

# Get MAE&MSE 

In [ ]:
baseline_mse = get_mse(baseline_sparse, valid_sparse_matrix)
baseline_mse
baseline_mae = get_mae(baseline_sparse, valid_sparse_matrix)
baseline_mae

In [ ]:
cf_als_mse = get_mse(cf_als_test, valid_sparse_matrix)
cf_als_mse
cf_als_mae = get_mae(cf_als_test, valid_sparse_matrix)
cf_als_mae 

cf_sgd_mse = get_mse(cf_sgd_sparse, valid_sparse_matrix)
cf_sgd_mse
cf_sgd_mae = get_mae(cf_sgd_sparse, valid_sparse_matrix)
cf_sgd_mae

In [ ]:
nlp_mse = get_mse(nlp_sparse, valid_sparse_matrix)
nlp_mse
nlp_mae = get_mae(nlp_sparse, valid_sparse_matrix)
nlp_mae

cb_tfidf_mse = get_mse(cb_tfidf_sparse, valid_sparse_matrix)
cb_tfidf_mse
cb_tfidf_mae = get_mae(cb_tfidf_sparse, valid_sparse_matrix)
cb_tfidf_mae

cb_lsi_mse = get_mse(cb_lsi_sparse, valid_sparse_matrix)
cb_lsi_mse
cb_lsi_mae = get_mae(cb_lsi_sparse, valid_sparse_matrix)
cb_lsi_mae

# Weight Each Model Based on MSE

In [ ]:
baseline = 1/ baseline_mse
cf_als = 1/ cf_als_mse
cf_sgd = 1/ cf_sgd_mse
nlp = 1/ nlp_mse
cb_tfidf = 1/ cb_tfidf_mse
cb_lsi = 1/cb_lsi_mse


all_ =   baseline + cf_als + cf_sgd + nlp + cb_tfidf + cb_lsi 

baseline = baseline/ all_
cf_als = cf_als/ all_
cf_sgd = cf_sgd/ all_
nlp = nlp / all_
cb_tfidf = cb_tfidf/ all_
cb_lsi = cb_lsi/ all_

# Get the Ensemble Matrix

In [ ]:
ensemble_matrix =  baseline * baseline_sparse \
+ cf_als * cf_als_sparse +  cf_sgd * cf_sgd_sparse \
+ nlp * nlp_sparse + cb_tfidf * cb_tfidf_sparse + cb_lsi * cb_lsi_sparse

ensemble_mse = get_mse(ensemble_matrix, valid_sparse_matrix)
ensemble_mse

ensemble_mae = get_mae(cf_sgd_sparse, valid_sparse_matrix)
ensemble_mae

np.save('../data/ensemble_matrix.npy', ensemble_matrix)

# Generate the full matrix of ensemble

In [ ]:
#Baseline model
baseline_long_full = pd.read_csv('../data/baseline_predict_df.csv')
baseline_long_full = baseline_long_full.loc[ : , ~baseline_long_full.columns.str.contains('Unnamed')]
baseline_long_full.head(2)
baseline_full_sparse = long_format_to_sparse(baseline_long_full, 'prediction')
baseline_full_sparse

# ALS
cf_als_sparse = np.load('../data/test_prediction_CF_ALS.npy')
cf_als_sparse
# SGD
cf_sgd_sparse = np.load('../data/test_prediction_CF_SGD.npy')
cf_sgd_sparse

# RA model
cb_ra_sparse = np.load('../data/pred_matrix.npy')
cb_ra_sparse
# NLP model
cb_nlp_sparse = np.load('../data/pred_matrix_nlp.npy')
cb_nlp_sparse

In [ ]:
nlp = 1/ nlp_mse
cb_tfidf = 1/ cb_tfidf_mse
cf_als = 1/ cf_als_mse
cf_sgd = 1/ cf_sgd_mse
baseline_ = 1/ baseline_mse_

all_ = baseline_  + cf_als +  cf_sgd + nlp + cb_tfidf 

baseline_ = baseline_/ all_
cf_als = cf_als/ all_
cf_sgd = cf_sgd/ all_
nlp = nlp/ all_
cb_tfidf = cb_tfidf/ all_

ensemble_full_matrix =  
baseline_ * baseline_full_sparse \
+cf_als * cf_als_sparse + cf_sgd * cf_sgd_sparse \
+ nlp * cb_ra_sparse + cb_tfidf * cb_nlp_sparse